<a href="https://colab.research.google.com/github/monicman/dataengineering/blob/master/Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install pyspark

In [0]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark import SparkFiles
import urllib
from urllib.request import urlopen   
import io
import requests
from zipfile import ZipFile
import os

In [0]:
spark = SparkSession.builder.getOrCreate()

def ingest_files(url,filename):
  spark.sparkContext.addFile(url)
  if url[-3:] == 'csv':
    return spark.read.csv(SparkFiles.get(filename), header=True)
  elif url[-3:] == 'zip':
    url = urlopen(url)
    output = open(filename, 'wb')    # note the flag:  "wb"        
    output.write(url.read())
    output.close()
    print("error here")
    df = pd.read_csv(filename,sep='\t')
    print("no actually here")
    return spark.createDataFrame(df)

In [0]:
spark = SparkSession.builder.getOrCreate()
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, DateType

def ingest_files(url,filename):
  spark.sparkContext.addFile(url)
  if url[-3:] == 'csv':
    return spark.read.csv(SparkFiles.get(filename), header=True)
  elif url[-3:] == 'zip':
    url = urlopen(url)
    output = open(filename, 'wb')    # note the flag:  "wb"        
    output.write(url.read())
    output.close()
    df = pd.read_csv(filename,sep='\t')
    df.iloc[:, 3:6] = df.iloc[:, 3:6].apply(pd.to_datetime, errors='coerce', format='%d/%m/%Y')
    myschema = StructType([StructField('REGISTER_NAME', StringType(), True),
                         StructField('BN_NAME', StringType(), True),
                         StructField('BN_STATUS', StringType(), True),
                         StructField('BN_REG_DT', DateType(), True),
                         StructField('BN_CANCEL_DT', DateType(), True),
                         StructField('BN_RENEW_DT', DateType(), True),
                         StructField('BN_STATE_NUM', StringType(), True),
                         StructField('BN_STATE_OF_REG', StringType(), True),
                         StructField('BN_ABN', DoubleType(), True)])
    return spark.createDataFrame(df, schema=myschema)

In [39]:
ipgod101 = 'https://data.gov.au/data/dataset/a4210de2-9cbb-4d43-848d-46138fefd271/resource/e5cbeafc-5fb3-4dfd-bd22-afe81b6ab1e1/download/ipgod101.csv'
ipgod102 = 'https://data.gov.au/data/dataset/a4210de2-9cbb-4d43-848d-46138fefd271/resource/846990df-db42-4ad7-bbd6-567fd37a2797/download/ipgod102.csv'  
ABN = 'https://data.gov.au/data/dataset/bc515135-4bb6-4d50-957a-3713709a76d3/resource/839cc783-876f-47a2-a70c-0fe606977517/download/business_names_201909.zip'

#staging_ipgod101 = ingest_files(ipgod101,'ipgod101.csv')
#staging_ipgod102 = ingest_files(ipgod102,'ipgod102.csv')
staging_abr = ingest_files(ABN,'zipFile.zip')

TypeError: ignored

In [15]:
ABN = 'https://data.gov.au/data/dataset/bc515135-4bb6-4d50-957a-3713709a76d3/resource/839cc783-876f-47a2-a70c-0fe606977517/download/business_names_201909.zip'
staging_abr.head(10)

[Row(REGISTER_NAME='BUSINESS NAMES', BN_NAME='         STEFAN ELECTRICAL TM.', BN_STATUS='Registered', BN_REG_DT=datetime.date(2014, 8, 6), BN_CANCEL_DT=None, BN_RENEW_DT=datetime.date(2020, 8, 6), BN_STATE_NUM='NaN', BN_STATE_OF_REG='NaN', BN_ABN=27474223588.0),
 Row(REGISTER_NAME='BUSINESS NAMES', BN_NAME='    Twinkle Toes Baby Hand and Feet Sculptures - North Melbourne', BN_STATUS='Registered', BN_REG_DT=datetime.date(2016, 8, 3), BN_CANCEL_DT=None, BN_RENEW_DT=datetime.date(2019, 6, 23), BN_STATE_NUM='NaN', BN_STATE_OF_REG='NaN', BN_ABN=29133895342.0),
 Row(REGISTER_NAME='BUSINESS NAMES', BN_NAME='   Nourishing Your World', BN_STATUS='Registered', BN_REG_DT=datetime.date(2017, 3, 13), BN_CANCEL_DT=None, BN_RENEW_DT=datetime.date(2020, 3, 13), BN_STATE_NUM='NaN', BN_STATE_OF_REG='NaN', BN_ABN=23850847348.0),
 Row(REGISTER_NAME='BUSINESS NAMES', BN_NAME='  colour expert painting and decorating service', BN_STATUS='Registered', BN_REG_DT=datetime.date(2013, 4, 23), BN_CANCEL_DT=None, 

In [16]:
staging_ipgod101.head(5)

[Row(australian_appl_no='2018904968', application_year='2018', sealing_year=None, applicant_count='1', australian='True', foreign='False', entity='False', non_entity='True', patent_status_type='FILED', patent_type='Provisional', primary_ipc_mark_value=None),
 Row(australian_appl_no='2017215797', application_year='2018', sealing_year=None, applicant_count='1', australian='False', foreign='True', entity='True', non_entity='False', patent_status_type='FILED', patent_type='Standard', primary_ipc_mark_value='E06B   9/00'),
 Row(australian_appl_no='2017216336', application_year='2018', sealing_year=None, applicant_count='1', australian='False', foreign='True', entity='True', non_entity='False', patent_status_type='FILED', patent_type='Standard', primary_ipc_mark_value='A61M  31/00'),
 Row(australian_appl_no='2017216324', application_year='2018', sealing_year=None, applicant_count='1', australian='False', foreign='True', entity='True', non_entity='False', patent_status_type='FILED', patent_ty

In [17]:
staging_ipgod102.head(5)

[Row(australian_appl_no='1904000001', ipa_id=None, country='ZZ', australian='False', entity='False', name='non-entity', cleanname='non-entity', corp_desg=None, state=None, postcode=None, lat=None, lon=None, sa2_code=None, sa2_name=None, lga_code=None, lga_name=None, gcc_name=None, elect_div=None, abn=None, acn=None),
 Row(australian_appl_no='1904000004', ipa_id=None, country='ZZ', australian='False', entity='False', name='non-entity', cleanname='non-entity', corp_desg=None, state=None, postcode=None, lat=None, lon=None, sa2_code=None, sa2_name=None, lga_code=None, lga_name=None, gcc_name=None, elect_div=None, abn=None, acn=None),
 Row(australian_appl_no='1904000006', ipa_id=None, country='ZZ', australian='False', entity='False', name='non-entity', cleanname='non-entity', corp_desg=None, state=None, postcode=None, lat=None, lon=None, sa2_code=None, sa2_name=None, lga_code=None, lga_name=None, gcc_name=None, elect_div=None, abn=None, acn=None),
 Row(australian_appl_no='1904000009', ipa_i

In [0]:
staging_ipgod101.createOrReplaceTempView('staging_ipgod101')
staging_ipgod102.createOrReplaceTempView('staging_ipgod102')
staging_abr.createOrReplaceTempView('staging_abr')


application =  spark.sql(""" SELECT DISTINCT 
                                      staging_ipgod101.australian_appl_no,
                                      staging_ipgod101.patent_status_type,
                                      staging_ipgod101.patent_type,
                                      staging_ipgod101.primary_ipc_mark_value,
                                      staging_ipgod102.ipa_id,
                                      staging_ipgod102.lat,
                                      staging_ipgod102.lon
                                      from staging_ipgod101 inner join staging_ipgod102
                                      on staging_ipgod101.australian_appl_no = staging_ipgod102.australian_appl_no
                                      where staging_ipgod102.australian = 'True' and staging_ipgod102.entity = 'True'
                                      order by staging_ipgod101.australian_appl_no,staging_ipgod102.ipa_id
                                      """)

region = spark.sql("""SELECT DISTINCT
                                       lat,
                                       lon,
                                       state,
                                       sa2_code,
                                       sa2_name,
                                       lga_code,
                                       lga_name,
                                       gcc_name,
                                       elect_div
                                   FROM staging_ipgod102
                                   where australian = 'True' and entity = 'True' and lat is not NULL and lon is not NULL
                                   order by lat,lon
                                   """) 

business = spark.sql("""SELECT DISTINCT
                                  staging_ipgod102.ipa_id,
                                  cast(staging_ipgod102.abn as int) as abn,
                                  staging_ipgod102.cleanname,
                                  staging_abr.BN_STATUS as business_registration_status,
                                  staging_abr.BN_REG_DT as business_registration_date,
                                  staging_abr.BN_REG_DT as business_cancel_date,
                                  staging_abr.BN_RENEW_DT as business_renew_date
                      from staging_ipgod102 left join staging_abr 
                                  on cast(staging_ipgod102.abn as int) = cast(staging_abr.BN_ABN as int)
                                  """)

business = spark.sql("""SELECT DISTINCT *
                                  from(select staging_ipgod102.ipa_id,
                                              cast(cast(abn as double) as int) as abn,
                                               staging_ipgod102.cleanname
                                        where  where australian = 'True' and entity = 'True' and lat is not NULL and lon is not NULL)
,
                                  staging_abr.BN_STATUS as business_registration_status,
                                  staging_abr.BN_REG_DT as business_registration_date,
                                  staging_abr.BN_REG_DT as business_cancel_date,
                                  staging_abr.BN_RENEW_DT as business_renew_date
                      from staging_ipgod102 left join staging_abr 
                                  on  cast(abn as double) = staging_abr.BN_ABN
                                  """)

In [0]:
business1 = spark.sql("""SELECT DISTINCT
                                  staging_ipgod102.ipa_id,
                                  cast(cast(abn as double) as int) as abn,
                                  staging_ipgod102.acn,
                                  staging_ipgod102.cleanname
                                  from staging_ipgod102  
                               where australian = 'True' and entity = 'True' and lat is not NULL and lon is not NULL
""")

In [117]:
business.head(50)

Py4JJavaError: ignored

In [0]:
region_table =  region.select("start_time").distinct() \
        .withColumn("hour", hour("start_time")).withColumn("day", dayofmonth("start_time")) \
        .withColumn("week", weekofyear("start_time")).withColumn("month", month("start_time")) \
        .withColumn("year", year("start_time")).withColumn("weekday", dayofweek("start_time"))
